In [4]:
import pandas as pd
import sqlite3

In [5]:
conn = sqlite3.connect('transformed-games.db')
df = pd.read_excel('../processed-data/transformed-games.xlsx', sheet_name='Sheet1')
df.to_sql('games', conn, index=False,if_exists="replace")
curr = conn.cursor()

C:\Users\Max Brenner\Anaconda3\lib\site-packages\pandas\core\generic.py:2663: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [6]:
df.head()

,Date,Season,Game ID,Visitor?,Team,Pts,Wins,Losses,Won,Rest Days,Starter 1,Starter 2,Starter 3,Starter 4,Starter 5,Starter Change,Distance Change,Altitude Change
0,Tue Nov 12 2019,19-20,0,True,Oklahoma City Thunder,85,4,7,False,2,Shai Gilgeous-Alexander,Chris Paul,Danilo Gallinari,Hamidou Diallo,Steven Adams,True,1072.355309,-492
1,Tue Nov 12 2019,19-20,0,False,Indiana Pacers,111,7,4,True,2,Aaron Holiday,T.J. Warren,Domantas Sabonis,Malcolm Brogdon,JaKarr Sampson,False,0.000000,0
2,Tue Nov 12 2019,19-20,1,True,New York Knicks,102,2,9,False,2,Julius Randle,RJ Barrett,Marcus Morris,Frank Ntilikina,Taj Gibson,False,1187.668805,562
3,Tue Nov 12 2019,19-20,1,False,Chicago Bulls,120,4,7,True,3,Wendell Carter,Zach LaVine,Lauri Markkanen,Tomáš Satoranský,Chandler Hutchison,False,0.000000,0
4,Wed Nov 13 2019,19-20,0,True,Memphis Grizzlies,119,4,7,True,2,Jae Crowder,Ja Morant,Jaren Jackson,Dillon Brooks,Jonas Valančiūnas,False,910.474441,480


In [7]:
def query_db(query):
    execute = curr.execute(query)
    cols = [column[0] for column in execute.description]
    results = pd.DataFrame.from_records(data=execute.fetchall(), columns=cols)
    return results

In [17]:
# Home vs Away Wins
query = \
'''
SELECT Team, SUM(Won) as "Number Wins",
(SELECT SUM(Won)*1.0 / COUNT(*) FROM games g2 WHERE g2.Team = g.Team AND g2."Visitor?" = 1 AND g2.Season is "19-20") as "Visitor Win Ratio",
(SELECT SUM(Won)*1.0 / COUNT(*) FROM games g2 WHERE g2.Team = g.Team AND g2."Visitor?" = 0 AND g2.Season is "19-20") as "Home Win Ratio"
FROM games g
WHERE Season is "19-20"
GROUP BY Team
ORDER BY "Number Wins" DESC
'''
wins_df = query_db(query)
wins_df

,Team,Number Wins,Visitor Win Ratio,Home Win Ratio
0,Milwaukee Bucks,39,0.842105,0.920000
1,Los Angeles Lakers,33,0.826087,0.700000
2,Toronto Raptors,33,0.761905,0.708333
3,Denver Nuggets,31,0.590909,0.782609
4,Los Angeles Clippers,30,0.560000,0.800000
5,Boston Celtics,29,0.523810,0.782609
6,Oklahoma City Thunder,29,0.695652,0.590909
7,Miami Heat,28,0.434783,0.857143
8,Utah Jazz,28,0.583333,0.736842
9,Dallas Mavericks,27,0.666667,0.541667


In [18]:
vis_better_home = 0
for r, row in wins_df.iterrows():
    if row['Visitor Win Ratio'] > row['Home Win Ratio']: vis_better_home += 1
'How many teams actually perform better when they are the visitor?: {} / {}'.format(vis_better_home, 30)

'How many teams actually perform better when they are the visitor?: 10 / 30'